In [1]:
import re
import time
import requests
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('celebrity_net_worth.csv')

In [3]:
df['Category'] = df.url.apply(lambda x: x.split('/')[4])
df['Name_Surname'] = df.url.apply(lambda x: x.split('/')[5].replace('-net-worth', '').replace('-', '_').strip())

In [4]:
del(df['Unnamed: 0'])
del(df['Last Updated:'])
del(df['Unnamed: 0.1'])
del(df['url'])

In [5]:
df.columns = [col.replace(" ", "_").replace(":", "") for col in df.columns] 

In [6]:
df.shape

(12938, 9)

In [7]:
df.isna().sum()

Net_Worth           16
Date_of_Birth     1483
Gender            1153
Height            5881
Profession        1392
Nationality       1550
Salary           12496
Category             0
Name_Surname         0
dtype: int64

In [8]:
df = df.set_index(['Name_Surname'])

# Net Worth

In [9]:
df.Net_Worth = df.Net_Worth.astype('str')

In [10]:
worths = []
for x in df.Net_Worth:
    try:
        w = int(re.findall('\d+', x.replace(" ", "").replace("sand", "000").replace("Mil", "000000").replace('Bil', '000000000'))[0])
    except:
        w = np.nan
    worths.append(w)

df.Net_Worth = worths

# Height

In [11]:
heights = []
for x in df.Height:
    try:
        h = float(re.findall('\(\d\.*\d*', x)[0][1:])
    except:
        h = np.nan
    heights.append(h)

df.Height = heights

# AGE

In [12]:
ages = []
for x in df.Date_of_Birth:
    try:
        if len(re.findall('\(\d*', x))>0:
            a = int(re.findall('\(\d*', x)[0][1:])
        else:
            a = np.nan
    except:
        a = np.nan
    ages.append(a)

In [13]:
df['Age'] = ages

## Alive/Dead

In [14]:
ad = []
for x in df.Date_of_Birth:
    try:
        if len(re.findall('\w+ \d*, \d\d\d\d', x)) > 0:
            d = re.findall('\w+ \d*, \d\d\d\d', x)[0].replace(',', '')
        else:
            d = np.nan
    except:
        d = np.nan
    ad.append(d)

In [15]:
df['Alive'] = ad

## BirthDate

In [16]:
dates = []
for x in df.Date_of_Birth:
    try:
        if len(re.findall('\w+ \d+, \d\d\d\d', x))>0:
            a = re.findall('\w+ \d+, \d\d\d\d', x)[0].replace(',', '')
        else:
            a = np.nan
    except:
        a = np.nan
    dates.append(a)

In [17]:
df.Date_of_Birth = dates

## Zodiac Sign

In [18]:
def zodiac_sign(month, day): 
      
    # checks month and date within the valid range of a specified zodiac 
    
    if month == 'Dec': 
        astro_sign = 'Sagittarius' if (day < 22) else 'Capricorn'
    elif month == 'Jan': 
        astro_sign = 'Capricorn' if (day < 20) else 'Aquarius'
    elif month == 'Feb': 
        astro_sign = 'Aquarius' if (day < 19) else 'Pisces'
    elif month == 'Mar': 
        astro_sign = 'Pisces' if (day < 21) else 'Aries'
    elif month == 'Apr': 
        astro_sign = 'Aries' if (day < 20) else 'Taurus'
    elif month == 'May': 
        astro_sign = 'Taurus' if (day < 21) else 'Gemini'
    elif month == 'Jun': 
        astro_sign = 'Gemini' if (day < 21) else 'Cancer'
    elif month == 'Jul': 
        astro_sign = 'Cancer' if (day < 23) else 'Leo'
    elif month == 'Aug': 
        astro_sign = 'Leo' if (day < 23) else 'Virgo'
    elif month == 'Sep': 
        astro_sign = 'Virgo' if (day < 23) else 'Libra'
    elif month == 'Oct': 
        astro_sign = 'Libra' if (day < 23) else 'Scorpio'
    elif month == 'Nov': 
        astro_sign = 'Scorpio' if (day < 22) else 'Sagittarius'

    return astro_sign

In [19]:
signs = []
for x in df.Date_of_Birth:
    try:
        x = zodiac_sign(x.split(' ')[0], int(x.split(' ')[1]))
    except:
        x = np.nan
    signs.append(x)

In [20]:
df['Zodiac_Sign'] = signs

# Salary

In [21]:
sal = []
for x in df.Salary:
    try:
        s = int(re.findall('\d+', x.replace(" ", "").replace("Thousand", "000").replace("Mil", "000000"))[0])
    except:
        s = np.nan
    sal.append(s)

In [22]:
df.Salary = sal

# Summin up..

In [23]:
df.isna().sum()

Net_Worth           16
Date_of_Birth     2285
Gender            1153
Height            5881
Profession        1392
Nationality       1550
Salary           12496
Category             0
Age               2269
Alive             2285
Zodiac_Sign       2285
dtype: int64

In [24]:
df.dropna().shape

(309, 11)

In [25]:
df.shape

(12938, 11)

In [26]:
df.to_csv('celebrity_cleaned.csv')